In [1]:
pip install torch torchvision matplotlib pillow


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 212.5/212.5 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 8.1/8.1 MB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 3.0 MB/s eta 0:00:00
     -------------------------------------- 134.9/134.9 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 194.4/194.4 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 222.0/222.0 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 4.1 MB/s eta 0:00:00
     -------------------------------------- 72.0/72.0 kB 493.8 kB/s eta 0:00:00
     -------------------------------------- 111.1/111.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: C:\Program Files\Python311\python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import copy

# Load and preprocess images
def load_image(img_path, max_size=400, shape=None):
    image = Image.open(img_path).convert('RGB')

    size = max_size if max(image.size) > max_size else max(image.size)
    if shape:
        size = shape

    in_transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    image = in_transform(image)[:3, :, :].unsqueeze(0)
    return image.to(device)

def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1, 2, 0)
    image = image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
    image = image.clip(0, 1)
    return image

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load images
content = load_image("samile_image.jpg").to(device)
style = load_image("download_imag_filter.jpeg", shape=content.shape[-2:]).to(device)

# Load pre-trained VGG19
vgg = models.vgg19(pretrained=True).features.to(device).eval()

# Freeze parameters
for param in vgg.parameters():
    param.requires_grad_(False)

# Define content and style layers
def get_features(image, model, layers=None):
    if layers is None:
        layers = {
            '0': 'conv1_1',
            '5': 'conv2_1',
            '10': 'conv3_1',
            '19': 'conv4_1',
            '21': 'conv4_2',  # content representation
            '28': 'conv5_1'
        }

    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

# Gram matrix for style
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

# Get features
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)

# Calculate Gram matrices
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# Initialize target image
target = content.clone().requires_grad_(True).to(device)

# Weights for layers
style_weights = {
    'conv1_1': 1.0,
    'conv2_1': 0.75,
    'conv3_1': 0.2,
    'conv4_1': 0.2,
    'conv5_1': 0.2
}
content_weight = 1  # alpha
style_weight = 1e6  # beta

# Optimizer
optimizer = optim.Adam([target], lr=0.003)

# Style Transfer Loop
steps = 2000
for i in range(steps):
    target_features = get_features(target, vgg)
    
    content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2']) ** 2)
    
    style_loss = 0
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        layer_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
        _, d, h, w = target_feature.shape
        style_loss += layer_loss / (d * h * w)

    total_loss = content_weight * content_loss + style_weight * style_loss
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if i % 400 == 0:
        print(f"Step {i}, Total loss: {total_loss.item():.4f}")

# Show the final result
plt.figure(figsize=(10, 5))
plt.imshow(im_convert(target))
plt.title("Stylized Image")
plt.axis("off")
plt.show()
